In [1]:
VALID_DATA = "zindi_data/ValDataset.csv"
DATA_PTH = "zindi_data/validation/prediction_cond-detr-50_THR1.000_IOU0.800_ID1740_P.csv"
THR_INF = 0.05
CLS_MAPPER = {"Trophozoite": 0, "WBC": 1, "NEG": 2}

In [2]:
import pandas as pd
import json
import os
from tqdm import tqdm

import numpy as np

In [3]:
data_pth = DATA_PTH

valid_data = pd.read_csv(VALID_DATA)

data = pd.read_csv(data_pth)
data.sample(3)

,image_id,x,y,w,h,category_id,score
15313,id_j8ri1iub1e.jpg,957.77,2343.83,84.78,96.42,Trophozoite,0.060
4594,id_147gpcegp6.jpg,3355.83,2735.43,99.99,117.47,Trophozoite,0.050
15685,id_4f9wdugdot.jpg,1172.22,2471.34,97.89,105.25,Trophozoite,0.061


In [4]:
valid_data.sample(3)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,width,height
456,id_ch6r0g46fr.jpg,Trophozoite,1.0,798,1358,951,1453,4032,3016
1393,id_xmq3awiu9f.jpg,Trophozoite,1.0,1290,1224,1394,1350,4160,3120
977,id_uhss5k1wwd.jpg,Trophozoite,1.0,2241,3662,2341,3762,4032,3016


In [5]:
valid_data = valid_data.rename(columns={
	"Image_ID": "image_id", "class": "category_id", "confidence": "score", "xmin": "x", "ymin": "y"
})

valid_data["w"] = valid_data["xmax"] - valid_data["x"]
valid_data["h"] = valid_data["ymax"] - valid_data["y"]


valid_data.sample(3)

,image_id,category_id,score,y,x,ymax,xmax,width,height,w,h
432,id_0o3fxfendl.jpg,Trophozoite,1.0,883,413,916,450,1920,1080,37,33
1698,id_6ujuefpy4m.jpg,Trophozoite,1.0,692,399,718,431,1920,1080,32,26
2168,id_ndhdv3wuvg.jpg,Trophozoite,1.0,658,991,700,1023,1920,1080,32,42


In [6]:
data["image_id"].nunique(), valid_data["image_id"].nunique()

(275, 275)

In [7]:
len(set(data["image_id"].unique()).intersection(valid_data["image_id"].unique()))

275

In [8]:
data["image_id"].value_counts().head(10)

image_id
id_dvhekvgy6v.jpg    95
id_qicgns95fp.jpg    94
id_wgijl6n37b.jpg    94
id_qa70t0m4yc.jpg    93
id_3xidcn77dw.jpg    91
id_0vaw9rsiro.jpg    91
id_15rb0dgq66.jpg    91
id_dzxgm7pfdb.jpg    90
id_bkc1sypwhe.jpg    90
id_w8xnbd5rvm.jpg    90
Name: count, dtype: int64

In [9]:
valid_data["image_id"].value_counts().head(10)

image_id
id_15rb0dgq66.jpg    70
id_zg9jc73sf7.jpg    48
id_zdg96srigj.jpg    42
id_i7nitsscaw.jpg    39
id_ndhdv3wuvg.jpg    39
id_z0i61ad0tq.jpg    38
id_ilmjjeqpcy.jpg    37
id_by6e6shi2z.jpg    36
id_q18tfhfneh.jpg    33
id_087dra2apu.jpg    33
Name: count, dtype: int64

In [10]:
data["score"].describe()

count    20773.000000
mean         0.150609
std          0.131796
min          0.032000
25%          0.066000
50%          0.103000
75%          0.176000
max          0.736000
Name: score, dtype: float64

In [11]:
data.isna().sum()

image_id       0
x              0
y              0
w              0
h              0
category_id    0
score          0
dtype: int64

In [12]:
len(valid_data[valid_data["category_id"] == "NEG"])

70

In [13]:
data.columns

Index(['image_id', 'x', 'y', 'w', 'h', 'category_id', 'score'], dtype='object')

In [14]:
valid_data.columns

Index(['image_id', 'category_id', 'score', 'y', 'x', 'ymax', 'xmax', 'width',
       'height', 'w', 'h'],
      dtype='object')

In [15]:
data.sample(5)

,image_id,x,y,w,h,category_id,score
11476,id_k50besa0sa.jpg,3148.75,2345.93,97.70,113.33,Trophozoite,0.055
16720,id_u0xldti3vu.jpg,1697.96,2151.57,66.28,74.67,Trophozoite,0.107
15018,id_5mtzjxoohq.jpg,2529.00,1912.40,67.45,82.66,Trophozoite,0.110
6465,id_49n2r2g61a.jpg,3620.08,1928.54,67.23,79.39,Trophozoite,0.174
9453,id_jiv444vkig.jpg,1038.13,385.01,70.09,85.05,Trophozoite,0.062


In [16]:
import torch

image_files: list[str] = valid_data["image_id"].unique().tolist()

def get_format_pred(file_id: str, data: pd.DataFrame, thr = THR_INF):
	pred = data[data["image_id"] == file_id].dropna(subset="x")
	pred = pred[pred["category_id"] != "NEG"]
	pred = pred[pred["score"] >= thr]
	if not len(pred):
		# return dict(
		# 	boxes=torch.empty(size=[0, 4], dtype=torch.float).cuda(),
		# 	scores=torch.empty(size=[0], dtype=torch.float).cuda(),
		# 	labels=torch.empty(size=[0,], dtype=torch.int).cuda(),
		# )
		return dict(
			boxes=torch.tensor([[0., 0., 0., 0.]], dtype=torch.float).cuda(),
			scores=torch.tensor([1.], dtype=torch.float).cuda(),
			labels=torch.tensor([CLS_MAPPER["NEG"]], dtype=torch.int).cuda(),
		)
	boxes = torch.tensor(pred[["x", "y", "w", "h"]].values, dtype=torch.float).cuda()
	scores = torch.tensor(pred["score"].values, dtype=torch.float).cuda()
	labels = torch.tensor([CLS_MAPPER[i] for  i in pred["category_id"].values], dtype=torch.int).cuda()
	return dict(boxes=boxes, scores=scores, labels=labels)


print("Prediction Extraction")
predictions = [
	get_format_pred(i, data) for i in image_files
]
print("Expected Extraction")
expected = [
	get_format_pred(i, valid_data) for i in image_files
]
# for i in expected:
# 	i.pop("scores")

Prediction Extraction
Expected Extraction


In [17]:
predictions[0]

{'boxes': tensor([[ 910.7900,  991.0100,  166.5600,  178.6400],
         [ 945.0100,  210.0700,   83.7900,   97.0900],
         [ 492.9300, 1768.1200,   81.4800,   89.0500],
         [1885.8101, 1384.0200,   85.4000,   93.7000],
         [1736.9100,  927.0000,   79.9600,   88.1100],
         [2719.1399, 1035.4100,   81.4300,   92.4200],
         [1858.9301, 1902.5000,   78.9800,   88.8100],
         [ 456.4000,  500.0200,   81.6700,   93.2000],
         [2827.1001, 1348.1899,   81.7400,   94.3100],
         [2286.0801,  920.9000,   79.9300,   89.4900],
         [3036.8999, 1299.1400,   78.1600,   92.6600],
         [1705.8800, 2489.0400,   79.6300,   88.7800],
         [1500.1300, 1147.1899,   79.1200,   89.4200],
         [ 507.3400, 1777.8400,   75.0400,   84.0000],
         [2994.5200, 1295.1300,   74.6600,   86.8900],
         [ 690.5200, 2230.1699,   84.2700,   97.2000],
         [  64.9800, 1817.8500,   83.0400,   99.6100],
         [2946.3401, 1256.0900,   74.8600,   84.6000],
 

In [ ]:
expected[0]

{'boxes': tensor([[1744.,  925.,   79.,  116.],
         [2278.,  904.,  116.,  116.],
         [3060., 1141.,  100.,   95.],
         [1881., 1913.,   95.,  106.],
         [1702., 2489.,   85.,   90.],
         [2674., 1501.,  100.,   85.],
         [3023., 1675.,  106.,  111.],
         [2848., 1353.,   90.,  106.],
         [ 470., 1765.,  116.,  122.],
         [2943., 1226.,   95.,  106.],
         [2933.,  671.,   95.,   85.],
         [1311.,  534.,   90.,  100.],
         [ 254., 2003.,  100.,  111.],
         [ 941.,  206.,   90.,  106.],
         [2949., 1041.,   90.,  106.],
         [1649., 2669.,   90.,   95.],
         [ 460.,  523.,   90.,   85.],
         [ 925.,  988.,  169.,  206.],
         [1887., 1395.,   95.,   90.],
         [3012.,  772.,  106.,  100.],
         [2743., 1046.,   74.,   85.]], device='cuda:0'),
 'scores': tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1.], device='cuda:0'),
 'labels': tensor([0, 

In [19]:
from torchmetrics.detection import MeanAveragePrecision

metric = MeanAveragePrecision(iou_type="bbox", box_format="xywh", class_metrics=True, extended_summary=False, average="macro")


metric.update(preds=predictions, target=expected)

from pprint import pprint
pprint(metric.compute())

{'classes': tensor([0, 1, 2], dtype=torch.int32),
 'map': tensor(0.0520),
 'map_50': tensor(0.1777),
 'map_75': tensor(0.0130),
 'map_large': tensor(0.1180),
 'map_medium': tensor(0.0882),
 'map_per_class': tensor([0.0472, 0.1088, 0.0000]),
 'map_small': tensor(0.0103),
 'mar_1': tensor(0.0310),
 'mar_10': tensor(0.1377),
 'mar_100': tensor(0.1754),
 'mar_100_per_class': tensor([0.1956, 0.3305, 0.0000]),
 'mar_large': tensor(0.3613),
 'mar_medium': tensor(0.2290),
 'mar_small': tensor(0.0859)}


In [20]:
expected[128]

{'boxes': tensor([[1277.,  127.,   85.,   70.],
         [1583.,  878.,   62.,   61.],
         [ 755.,  815.,   59.,   64.],
         [ 929.,  165.,   38.,   34.],
         [ 864.,  375.,   96.,   79.],
         [1430.,   66.,   26.,   25.],
         [ 473.,  513.,   32.,   38.],
         [1110.,  131.,   28.,   28.],
         [ 908.,  291.,   64.,   59.],
         [1394.,  600.,   28.,   26.],
         [1226.,  269.,   32.,   34.],
         [1432.,  110.,   30.,   32.],
         [ 653.,  815.,   30.,   32.],
         [1131.,  161.,   62.,   81.]], device='cuda:0'),
 'scores': tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        device='cuda:0'),
 'labels': tensor([1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1], device='cuda:0',
        dtype=torch.int32)}

In [ ]:
predictions[128]

{'boxes': tensor([[ 755.6900,  797.6800,   59.9800,   61.4000],
         [1581.4500,  858.6600,   57.6100,   56.0900],
         [ 912.1600,  285.3500,   59.6700,   55.9700],
         [1279.9800,  131.9200,   81.8900,   63.6500],
         [ 862.1300,  984.0800,   30.6100,   31.6300],
         [ 761.8100,  298.5600,   35.2600,   31.2000],
         [ 710.3000,  302.0900,   31.9100,   29.8700],
         [ 629.2100,  410.2700,   34.2000,   34.3000],
         [ 858.2400,  364.3400,   84.7000,   76.4100],
         [ 689.8100,  628.0900,   57.1800,   49.2600],
         [ 763.3500,  304.9200,   32.4700,   29.9900],
         [1392.6500,  587.8800,   30.1000,   28.3800],
         [1129.4100,  170.8800,   58.2000,   68.4300],
         [ 738.9700,  141.0600,   61.3800,   57.1100],
         [ 655.7400,  797.8900,   31.6300,   30.6600],
         [1108.7600,  128.8600,   31.9600,   29.9600],
         [1222.3000,  266.1300,   30.7700,   29.1900],
         [1166.5100,  587.1800,   75.3800,   60.8300],
 

In [22]:
from tqdm import tqdm

thrs = np.linspace(0.05, .95, num=15)
best_score = 0
best_thr = 0

for thr in tqdm(thrs):
	metric = MeanAveragePrecision(iou_type="bbox", box_format="xywh", class_metrics=True, extended_summary=False)
	predictions = [
		get_format_pred(i, data, thr) for i in image_files
	]
	expected = [
		get_format_pred(i, valid_data, thr) for i in image_files
	]
	metric.update(preds=predictions, target=expected)
	scores = metric.compute()
	if scores["map_50"] > best_score:
		best_score = scores["map_50"]
		best_thr = thr
		print(best_thr, best_score)

  7%|▋         | 1/15 [00:07<01:47,  7.66s/it]

0.05 tensor(0.1777)


 67%|██████▋   | 10/15 [00:36<00:12,  2.59s/it]

In [23]:
print(best_thr, best_score)

0.05 tensor(0.3330)


In [24]:
thrs

array([0.05      , 0.11428571, 0.17857143, 0.24285714, 0.30714286,
       0.37142857, 0.43571429, 0.5       , 0.56428571, 0.62857143,
       0.69285714, 0.75714286, 0.82142857, 0.88571429, 0.95      ])

In [25]:
thr = .0

metric = MeanAveragePrecision(iou_type="bbox", box_format="xywh", class_metrics=True, extended_summary=False)
predictions = [
    get_format_pred(i, data, thr) for i in image_files
]
expected = [
    get_format_pred(i, valid_data, thr) for i in image_files
]
metric.update(preds=predictions, target=expected)
scores = metric.compute()

In [26]:
scores

{'map': tensor(0.1256),
 'map_50': tensor(0.3346),
 'map_75': tensor(0.0666),
 'map_small': tensor(0.0238),
 'map_medium': tensor(0.1650),
 'map_large': tensor(0.2604),
 'mar_1': tensor(0.0450),
 'mar_10': tensor(0.1720),
 'mar_100': tensor(0.2083),
 'mar_small': tensor(0.0844),
 'mar_medium': tensor(0.2911),
 'mar_large': tensor(0.3808),
 'map_per_class': tensor([0.0568, 0.3200, 0.0000]),
 'mar_100_per_class': tensor([0.2023, 0.4226, 0.0000]),
 'classes': tensor([0, 1, 2], dtype=torch.int32)}